# Swap

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) and the Refinitiv Data Libraries.

##### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

#### Import the library and connect to the Refinitv Data Libraries

In [1]:
import refinitiv.data as rd
from refinitiv.data.content.ipa.financial_contracts import swap

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x103e5d5e0 {name='workspace'}>

### Valuing a Vanilla IRS

This sample value an existing swap defined leg by leg for a complete control of its term and conditions. The 'fields' requested are:
- the market value in cash (CleanMarketValueInDealCcy, MarketValueInDealCcy, AccruedAmountInDealCcy)
- their equivalent in in percent (AccruedPercent,CleanPricePercent,DirtyPricePercent,CleanPricePercent)
- the PV01 (PV01Bp,PV01AmountInDealCcy)
- the names of the zero-coupon curves used (DiscountCurveName,ForwardCurveName)
- the error information (ErrorCode,ErrorMessage)


In [3]:
response = swap.Definition(
    instrument_tag="IRS-EURIBOR-5Y",
    start_date="2020-01-28",
    tenor="5Y",
    legs=[
        swap.LegDefinition(
            direction=swap.Direction.PAID,
            notional_ccy="EUR",
            interest_payment_frequency=swap.Frequency.ANNUAL,
            interest_type=swap.InterestType.FIXED,
            interest_calculation_method="Dcb_30_360",
            fixed_rate_percent=0.5
        ),
        swap.LegDefinition(
            direction=swap.Direction.RECEIVED,
            notional_ccy="EUR",
            interest_payment_frequency=swap.Frequency.SEMI_ANNUAL,
            index_name="EURIBOR",
            interest_type=swap.InterestType.FLOAT,
            interest_calculation_method="Dcb_Actual_360",
            index_tenor="6M",
        ),
    ],
    pricing_parameters=swap.PricingParameters(
        valuation_date="2020-10-20T00:00:00Z"
    ),
    fields=["InstrumentTag",
            "MarketValueInDealCcy",
            "DirtyPricePercent",
            "FixedRatePercent", "SpreadBp",
            "PV01Bp",
            "PV01AmountInDealCcy",
            "DiscountCurveName",
            "ForwardCurveName",
            "ErrorCode",
            "ErrorMessage"],
).get_data()

response.data.df

,InstrumentTag,MarketValueInDealCcy,DirtyPricePercent,FixedRatePercent,SpreadBp,PV01Bp,PV01AmountInDealCcy,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,IRS-EURIBOR-5Y,25290.342903,2.529034,0.5,<NA>,0.058033,5.803305,EUR - Swap vs 6M Euribor,<NA>,,
1,IRS-EURIBOR-5Y,25290.342903,2.529034,<NA>,101.695705,-4.003007,-400.300656,EUR - Swap vs 6M Euribor,EUR - Swap vs 6M Euribor,,


In [4]:
rd.close_session()